In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# Import the commmly used libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Make sure the ipython notebook will display the figures 
%matplotlib inline

In [ ]:
# Load the data
data = pd.read_csv ("../input/creditcard.csv")

# Display the first few lines of the credit card data
data.head()

In [ ]:
# Display the column names in the data
data.columns

It seems that there are 31 columns in the data

In [ ]:
count_classes = pd.value_counts(data['Class'], sort = True).sort_index()
count_classes

Apparently, most data here are in the "normal" class, or to say that most transaction data are normal. Only a few transaction data points are conisdered to be abnormal, or fraut. Thus, this is an unbalanced data sets, are therefor challenging for the maching learning prediction, especially for accuracy. Thefore, we need to adjust the amount of normal class data and the abnormal class data, to make them balanaced, by either increasing the amounf of the abnormal class data using the technique called over sampling, or by decreasing the amount of the normal class data using the techque called under sampling. 

Here I chose the latter technique by seleting some data points out of normal class data set.

In [ ]:
# First of all, we need to decide which columns are our features and which is/are variant(s) 
# we need to predict
fcol = data.ix[:, data.columns != 'Class']
rcol = data.ix[:, data.columns == 'Class']

# To begin with the data selecting procedure, we need to find out the indices of both the normal
abnormal_indices = np.array(data[data.Class == 1].index)
normal_indices = np.array(data[data.Class == 0].index)

#Find out the number of abnormal data entries 
n_abnormal = len(data[data.Class == 1])


# Then, we can randomly select the normal data entries' indices
random_normal_indices = np.random.choice(normal_indices, n_abnormal, replace = False)
len(random_normal_indices)

# Now we can merge the selected normal and the original normal data entries
under_sample_indices = np.concatenate([abnormal_indices,random_normal_indices])
under_sample_data = data.ix[under_sample_indices,:]

# Based on this under samppled data set, we can indicate our feature colummns and result columns
fcol_under = under_sample_data.ix[:, data.columns != 'Class']
rcol_under = under_sample_data.ix[:, data.columns == 'Class']

# Double check the data size for both normal and abnormal class in this generated data set
print("Number of normal transactions", len(under_sample_data.Class == 0))
print("Number of abnormal transactions", len(under_sample_data.Class == 1))



Now that we have create a balanced dataset, we can prepare our training and testing datasets by spliting.

In [ ]:
# Import the spliting package
from sklearn.cross_validation import train_test_split

# Prepare the training and testing data sets from the original whole data
f_train, f_test,r_train, r_test = train_test_split(fcol, rcol, test_size = 0.33, random_state = 0)

# Prepare the training and testing data sets from the under sampled data
f_under_train, f_under_test,r_under_train, r_under_test = train_test_split(fcol_under, rcol_under, test_size = 0.33, random_state = 0) 


Now we can use the logistic regression classfier to train our data from the original and the under sampled conditions.

In [ ]:
# Import the logist regression, cross_validation and prediction metrics packages.
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 

In [ ]:
# Define a kfold score function to diplay the recall score with different choices of C parameter
def Kfold_scores(f_train_data,r_train_data):
    fold = KFold(len(r_train_data),5,shuffle=False) 

    # Different C parameters
    c_param_range = [0.01,0.1,1,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold,start=1):

            # Call the logistic regression model with a certain C parameter
            lr = LogisticRegression(C = c_param, penalty = 'l1')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            lr.fit(f_train_data.iloc[indices[0],:],r_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            r_pred_undersample = lr.predict(f_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            recall_acc = recall_score(r_train_data.iloc[indices[1],:].values,r_pred_undersample)
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.ix[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

In [ ]:
best_c = Kfold_scores(f_under_train, r_under_train)

In [ ]:
# Since we found the best C parameter is 10.0, now that we can try to find out the recall value 
# using the such C 
lr = LogisticRegression(C = best_c, penalty = 'l1')
lr.fit(f_under_train,r_under_train.values.ravel())
r_pred_undersample = lr.predict(f_under_test.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(r_under_test,r_pred_undersample)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))


In [ ]:
# We can do the similar thing using the original whold data
lr.fit(f_train,r_train.values.ravel())
r_pred_Wholesample = lr.predict(f_test.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(r_test,r_pred_Wholesample)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))


Apparently, our under sampling data generating pays off!